In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [6]:
# Imports
from Models import ModelUtil
import importlib
from Data import Preprocessing
from sklearn.model_selection import train_test_split

In [7]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering()

c:\Users\Trent\Desktop\Senior Design\shifting_model\Data\DataUtil.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PitcherThrows"] = df["PitcherThrows"].map({"Left":1, "Right":2, "Both":3})


In [8]:
# 2) Training Models
Y = infieldDataFrame["FieldSlice"]
X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "ZoneSpeed", "PlateLocHeight", "PlateLocSide"]]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)

In [4]:
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 25
max_features = 10
max_leaf_nodes = 55
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)
#print(result)

training decision tree model...
done!
getting statistics...

Decisions Tree Data Splits: train=0.75, test=0.25
Decision Tree Depth: 11
Decision Tree Number of Leaves: 55
Decision Tree's Accuracy Score for Predicting on Training Data: 0.6813
Decision Tree's Accuracy Score for Predicting on Test Data: 0.2243

Decision Tree Overall Average Probabilities
-------------------------------------
Section 1: 26.18%
Section 2: 22.94%
Section 3: 20.52%
Section 4: 20.48%
Section 5: 9.87%
Decision Tree Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		62		59
2		90		59
3		74		89
4		65		71
5		29		42
Amount Correct: 218
Amount Incorrect: 102
Decision Tree Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		19		26
2		29		22
3		25		28
4		24		20
5		10		11
Amount Correct: 24
Amount Incorrect: 83
done!
(DecisionTreeClassifier(class_weight='balanced', max_depth=25, max_features=10,

c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:68: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\laure\Documents\Auburn\Senior Design\shifting_model\Models\ModelUtil.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of p

In [12]:
# b) Naive Bayes

var_smoothing = 1e-9
result = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)
test = True
print("Training Accuracy: ")
print(result[1])
print("Testing Accuracy: ")
print(result[2])

training Naive Bayes model...
done!
getting statistics...
done!
Training Accuracy: 
0.39375
Testing Accuracy: 
0.2803738317757009


In [16]:
# c)Logistic Regression
lr = 0.8
e = 100
result = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)
print("Training Accuracy: ")
print(result[1])
print("Testing Accuracy: ")
print(result[2])

training logistic regression model...
done!
getting statistics...
done!
Training Accuracy: 
0.340625
Testing Accuracy: 
0.2523364485981308


C:\Users\Trent\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
result = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)
print("Training Accuracy: ")
print(result[1])
print("Testing Accuracy: ")
print(result[2])

training SVM model...


InvalidParameterError: The 'gamma' parameter of SVC must be a str among {'scale', 'auto'} or a float in the range [0.0, inf). Got 'rbf' instead.

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements